In [ ]:
from typing import Union, Literal
import time
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [ ]:
# ## TODO: finish implementation

# class AcceleratedKMeans:
#     """
#     Perform AcceleratedKMeans clustering on a dataset.
#     """

#     def __init__(self,
#                  algorithm : Literal['lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan', 'binary-hartigan'] = 'lloyd',
#                  init : Literal['random', 'random-data', 'k-means++', 'maximin', 'greedy-k-means++'] = 'random',
#                  seed : Union[int, None] = None):
#         """
#         Initialize the KMeans object.

#         Parameters
#         ----------
#         algorithm : {'elkan', 'acc-hartigan'}
#             Algorithm to use. Either 'elkan' or 'acc-hartigan'

#         init : {'random', 'random-data', 'k-means++', 'maximin', 'greedy-k-means++'}
#             Initialization method. Either 'random' or 'random-data' or 'k-means++' or 'maximin' or 'greedy-k-means++'

#         seed : int
#             Seed for random generator
#         """

#         assert algorithm in ['elkan', 'acc-hartigan'], "algorithm must be either 'elkan' or 'acc-hartigan'"
#         assert init in ['random', 'random-data', 'k-means++', 'maximin', 'greedy-k-means++'], "init must be either 'random', 'random-data', 'k-means++', 'maximin', or 'greedy-k-means++'"
#         assert seed is None or isinstance(seed, int), "seed must be an int or None"

#         self.algorithm = algorithm
#         self.init = init
#         self.seed = seed

#         self.data = None
#         self.k = None
#         self.n_samples = None
#         self.centroids = None
#         self.y_pred = None
#         self.tol = 1e-4

#         self.l = None
#         self.u = None

#         self.safe_iterations = 0
#         self.iterations = 0
#         self.norm_calculations = 0

#         self._distance_cache = None


#     def fit(self, data : np.ndarray, k : int, debug : int = 0):
#         """
#         Fit the model to the data.

#         Parameters
#         ----------
#         data : np.ndarray
#             nxd DataFrame of n samples with d features
#         k : int
#             Number of clusters
#         debug : int
#             Debug level (0: no debug, 1: some debug, 2: all debug)

#         Returns
#         -------
#         np.ndarray
#             Array of shape (k, d) with cluster centroids
#         np.ndarray
#             Array of length n with cluster assignments for each sample
#         """

#         assert isinstance(data, np.ndarray), "data must be a numpy array"
#         assert len(data.shape) == 2, "data must be a 2D array"
#         assert isinstance(k, int), "k must be an int"
#         assert 0 < k <= len(data), "k must be at least 0 and at most the number of samples"
#         assert isinstance(debug, int) or debug, "debug must be an int"

#         self.data = data
#         self.k = k
#         self.n_samples = data.shape[0]
#         self.l = np.zeros((self.n_samples, self.k))
#         self.u = np.zeros((self.n_samples))

#         np.random.seed(self.seed)

#         # initialize centroids
#         self._init_centroids(debug)
#         debug and print('initial centroids:\n', self.centroids)
#         debug and print('initial y_pred:', self.y_pred)

#         if self.algorithm == 'elkan':
#             self._elkan(debug = debug)
#         elif self.algorithm == 'acc-hartigan':
#             self._acc_hartigan(debug = debug)
        
#         debug and print('final centroids:\n', self.centroids)
#         debug and print('final y_pred:', self.y_pred)


#     def _init_centroids(self, debug=0):
#         """
#         Initialize the centroids.
#         """

#         v = 1

#         if self.init == 'random':

#             if v == 1:
#                 # choose k random data points as initial centroids
#                 idx = np.random.choice(self.n_samples, self.k, replace=False)
#                 self.centroids = self.data[idx]
#                 self.y_pred = self._assign_clusters(debug=debug>1)

#             elif v == 2:
#                 # choose k random data points as initial centroids
#                 idx = np.random.choice(self.n_samples, self.k, replace=False)
#                 self.centroids = self.data[idx]

#                 self._distance_cache = cdist(self.data, self.centroids, metric='sqeuclidean')
#                 self.y_pred = self._assign_clusters(debug=debug>1)

#         elif self.init == 'random-data':
#             return NotImplemented
            
#             # V1
#             # assign each data point to a random cluster
#             clusters = np.random.choice(self.k, self.n_samples)

#             # check that at least one point is assigned to each cluster
#             while len(set(clusters)) < self.k:
#                 clusters = np.random.choice(self.k, self.n_samples)
#             self.y_pred = clusters
#             self.centroids = self._move_centroids(None, debug > 1)

            
#             # V2
#             n_samples, n_features = self.data.shape
#             # Assign each data point to a random cluster
#             self.y_pred = np.random.choice(self.k, n_samples)
#             # Ensure at least one point is assigned to each cluster
#             unique_clusters = np.unique(self.y_pred)
#             if len(unique_clusters) < self.k:
#                 for i in range(self.k):
#                     if i not in unique_clusters:
#                         self.y_pred[np.random.randint(0, n_samples)] = i
            
#             self.centroids = np.zeros((self.k, n_features))
#             # Calculate centroids efficiently
#             for i in range(self.k):
#                 mask = self.y_pred == i
#                 if np.any(mask):
#                     self.centroids[i] = np.mean(self.data[mask], axis=0)
#                 else:
#                     # Handle empty cluster
#                     self.centroids[i] = self.data[np.random.randint(0, n_samples)]

#         elif self.init == 'k-means++':
            
#             if v == 1:
#                 # choose first centroid randomly
#                 centroids = np.zeros((self.k, self.data.shape[1]))
#                 centroids[0] = self.data[np.random.choice(self.n_samples, 1, replace=False)[0]]
#                 debug and print('centroids:\n', centroids)

#                 # iterate over remaining k-1 centroids
#                 for i in range(1, self.k):
#                     debug and print('iteration', i)

#                     # calculate squared distance of each point to closest centroid
#                     dist = np.min(np.linalg.norm(self.data[:, np.newaxis] - centroids[:i], axis=2) ** 2, axis=1)

#                     # probabilities are given by the normalized distance squared
#                     probs = dist / dist.sum()
#                     debug and print('probs:', probs)

#                     # choose next centroid randomly based on cumulated probabilities
#                     j = np.random.choice(self.n_samples, p=probs)

#                     centroids[i] = self.data[j]
#                     debug and print('centroids:\n', centroids)

#                 self.centroids = centroids
#                 self.y_pred = self._assign_clusters(debug=debug>1)

#             elif v == 2:
#                 # choose first centroid randomly
#                 self.centroids = np.zeros((self.k, self.data.shape[1]))
#                 first_idx = np.random.choice(self.n_samples)
#                 self.centroids[0] = self.data[first_idx].copy()

#                 debug and print('centroids:\n', self.centroids)

#                 # initialize min_distances with infinity
#                 min_distances = np.full(self.n_samples, np.inf)

#                 # iterate over remaining k-1 centroids
#                 for i in range(1, self.k):
#                     debug and print('iteration', i)

#                     # calculate squared distance of each point to closest centroid (only calculate for new centroids)
#                     new_distances = np.sum((self.data - self.centroids[i-1])**2, axis=1)
#                     np.minimum(min_distances, new_distances, out=min_distances)

#                     # probabilities are given by the normalized distance squared
#                     probs = min_distances / np.sum(min_distances)
#                     debug and print('probs:', probs)

#                     # choose next centroid randomly based on cumulated probabilities
#                     j = np.random.choice(self.n_samples, p=probs)

#                     self.centroids[i] = self.data[j].copy()
#                     debug and print('centroids:\n', self.centroids)

#                 self._distance_cache = cdist(self.data, self.centroids, metric='sqeuclidean')
#                 self.y_pred = np.argmin(self._distance_cache, axis=1)

#         elif self.init == 'maximin':

#             if v == 1:
#                 # choose first centroid randomly
#                 centroids = np.zeros((self.k, self.data.shape[1]))
#                 centroids[0] = self.data[np.random.choice(self.n_samples, 1, replace=False)[0]]
#                 debug and print('centroids:\n', centroids)

#                 # iterate over remaining k-1 centroids
#                 for i in range(1, self.k):
#                     debug and print('iteration', i)

#                     # calculate squared distance of each point to closest centroid
#                     dist = np.min(np.linalg.norm(self.data[:, np.newaxis] - centroids[:i], axis=2) ** 2, axis=1)

#                     # choose next centroid as the point with the maximum distance to the closest centroid
#                     centroids[i] = self.data[np.argmax(dist)]
#                     debug and print('centroids:\n', centroids)

#                 self.centroids = centroids
#                 self.y_pred = self._assign_clusters(debug=debug>1)

#             elif v == 2:
#                 # choose first centroid randomly
#                 self.centroids = np.zeros((self.k, self.data.shape[1]))
#                 first_idx = np.random.choice(self.n_samples)
#                 self.centroids[0] = self.data[first_idx].copy()

#                 # initialize min_distances with infinity
#                 min_distances = np.full(self.n_samples, np.inf)

#                 # iterate over remaining k-1 centroids
#                 for i in range(1, self.k):
#                     debug and print('iteration', i)

#                     # calculate squared distance of each point to closest centroid (only calculate for new centroids)
#                     new_distances = np.sum((self.data - self.centroids[i-1])**2, axis=1)
#                     np.minimum(min_distances, new_distances, out=min_distances) 

#                     # choose next centroid as the point with the maximum min_distance to the closest centroid
#                     self.centroids[i] = self.data[np.argmax(min_distances)].copy()
#                     debug and print('centroids:\n', self.centroids)

#                 self._distance_cache = cdist(self.data, self.centroids, metric='sqeuclidean')
#                 self.y_pred = np.argmin(self._distance_cache, axis=1)

#         elif self.init == 'greedy-k-means++':

#             if v == 1:
#                 # TODO: this might be adapted
#                 trials = 2 + int(np.log(self.k)) 

#                 # choose first centroid randomly
#                 centroids = np.zeros((self.k, self.data.shape[1]))
#                 f = np.random.choice(self.n_samples)
#                 centroids[0] = self.data[f]
#                 debug and print('centroids:\n', centroids)

#                 # iterate over remaining k-1 centroids
#                 for i in range(1, self.k):
#                     debug and print('iteration', i)

#                     # calculate squared distance of each point to closest centroid
#                     dist = np.min(np.linalg.norm(self.data[:, np.newaxis] - centroids[:i], axis=2) ** 2, axis=1)

#                     # probabilities are given by the normalized distance squared
#                     probs = dist / dist.sum()
#                     debug and print('probs:', probs)

#                     # choose next centroid randomly based on cumulated probabilities
#                     candidate_j = np.random.choice(self.n_samples, size=trials, p=probs)
#                     debug and print('candidate_j:', candidate_j)

#                     # calculate the cost of each trial
#                     costs = np.zeros(trials)
#                     for t in range(trials):
#                         centroids[i] = self.data[candidate_j[t]]
#                         costs[t] = self._tot_cluster_cost(centroids, self._assign_clusters(specific_centroids=centroids), debug=debug>1)
#                     debug and print('costs:', costs)

#                     # choose the trial with the lowest cost
#                     j = candidate_j[np.argmin(costs)]

#                     centroids[i] = self.data[j]
#                     debug and print('centroids:\n', centroids)

#                 self.centroids = centroids
#                 self.y_pred = self._assign_clusters(debug=debug>1)

#             elif v == 2:
#                 # V2
#                 # TODO: this might be adapted
#                 trials = 2 + int(np.log(self.k)) 

#                 # choose first centroid randomly
#                 self.centroids = np.zeros((self.k, self.data.shape[1]))
#                 first_idx = np.random.choice(self.n_samples)
#                 self.centroids[0] = self.data[first_idx].copy()

#                 # initialize min_distances with infinity
#                 min_distances = np.full(self.n_samples, np.inf)

#                 # iterate over remaining k-1 centroids
#                 for i in range(1, self.k):
#                     debug and print('iteration', i)

#                     # calculate squared distance of each point to closest centroid (only calculate for new centroids)
#                     new_distances = np.sum((self.data - self.centroids[i-1])**2, axis=1)
#                     np.minimum(min_distances, new_distances, out=min_distances)

#                     # probabilities are given by the normalized distance squared
#                     probs = min_distances / np.sum(min_distances)
#                     debug and print('probs:', probs)

#                     # choose next centroid randomly based on cumulated probabilities
#                     candidate_j = np.random.choice(self.n_samples, size=trials, p=probs)
#                     debug and print('candidate_j:', candidate_j)

#                     # calculate the cost of each trial
#                     best_cost = np.inf
#                     best_idx = -1

#                     # costs = np.zeros(trials)
#                     for t in range(trials):
#                         temp_centroids = self.centroids[:i].copy()
#                         temp_centroids = np.vstack([temp_centroids, self.data[t]])

#                         distances = cdist(self.data, temp_centroids, metric='sqeuclidean')
#                         assignments = np.argmin(distances, axis=1)

#                         # calculate cost
#                         cost = 0
#                         for j in range(i+1):
#                             mask = assignments == j
#                             if np.any(mask):
#                                 cost += np.sum(distances[mask, j])

#                         if cost < best_cost:
#                             best_cost = cost
#                             best_idx = t
                    
#                     self.centroids[i] = self.data[best_idx].copy()

#                     debug and print('centroids:\n', self.centroids)

#                 self._distance_cache = cdist(self.data, self.centroids, metric='sqeuclidean')
#                 self.y_pred = np.argmin(self._distance_cache, axis=1)

    
#     def dist(self, x, y):
#         """
#         Compute the squared Euclidean distance between two points.

#         Parameters
#         ----------
#         x : np.ndarray
#             First point
#         y : np.ndarray
#             Second point

#         Returns
#         -------
#         float
#             Squared Euclidean distance between x and y
#         """
#         self.norm_calculations += 1/(self.k*self.n_samples)
#         return np.sum((x - y)**2)

#     # chat version
#     def _elkan(self, debug=0):
#         """
#         Elkan's algorithm for k-means clustering.
#         """
#         debug and print("\nRunning Elkan's algorithm...")

#         n, k = self.data.shape[0], self.k
#         # Initialize bounds and distances
#         upper_bounds = np.full((n,), np.inf)           # u[i]
#         lower_bounds = np.zeros((n, k))                # l[i][j]
#         center_dists = np.zeros((k, k))                # d[c][j]
#         half_center_dists = np.zeros((k, k))           # s[c][j]

#         old_y_pred = np.full(n, -1, dtype=int)

#         # Precompute initial assignments
#         self.y_pred = self._assign_clusters()
        
#         for i in range(n):
#             upper_bounds[i] = np.linalg.norm(self.data[i] - self.centroids[self.y_pred[i]])
#             self.norm_calculations += 1/(self.k*self.n_samples)

#         while True:
#             debug and print("New iteration")
#             self.iterations += 1
#             np.copyto(old_y_pred, self.y_pred)
#             old_centroids = self.centroids.copy()

#             # Step 1: compute distances between all pairs of centroids
#             for c in range(k):
#                 for j in range(k):
#                     if c != j:
#                         dist = np.linalg.norm(self.centroids[c] - self.centroids[j])
#                         self.norm_calculations += 1/(self.k*self.n_samples)
#                         center_dists[c][j] = dist
#                         half_center_dists[c][j] = dist / 2.0

#             # Step 2: move centroids
#             self.centroids = self._move_centroids(None, debug > 1)

#             # Step 3: compute centroid movement distances
#             centroid_shifts = np.linalg.norm(self.centroids - old_centroids, axis=1)
#             self.norm_calculations += self.k / (self.k*self.n_samples)

#             # Step 4: update upper and lower bounds
#             for i in range(n):
#                 upper_bounds[i] += centroid_shifts[self.y_pred[i]]
#                 for j in range(k):
#                     lower_bounds[i][j] = max(lower_bounds[i][j] - centroid_shifts[j], 0)

#             # Step 5: reassign points
#             for i in range(n):
#                 c = self.y_pred[i]
#                 skip = False
#                 for j in range(k):
#                     if j == c:
#                         continue
#                     if upper_bounds[i] <= lower_bounds[i][j]:
#                         continue
#                     if upper_bounds[i] <= half_center_dists[c][j]:
#                         continue

#                     # Otherwise, need to compute actual distance
#                     if upper_bounds[i] == np.inf:
#                         dist_to_c = np.linalg.norm(self.data[i] - self.centroids[c])
#                         self.norm_calculations += 1/(self.k*self.n_samples)
#                     else:
#                         dist_to_c = upper_bounds[i]
#                     dist_to_j = np.linalg.norm(self.data[i] - self.centroids[j])
#                     self.norm_calculations += 1/(self.k*self.n_samples)

#                     lower_bounds[i][j] = dist_to_j

#                     if dist_to_j < dist_to_c:
#                         c = j
#                         dist_to_c = dist_to_j
#                         upper_bounds[i] = dist_to_c

#                 self.y_pred[i] = c

#             if np.array_equal(old_y_pred, self.y_pred):
#                 break


#     # # v2
#     # def _elkan(self, debug=0):
#     #     """
#     #     Elkan's algorithm for k-means clustering, optimized with triangle inequality bounds.
#     #     """

#     #     # Initialization
#     #     r = np.full(self.n_samples, True)                      # All upper bounds need to be recomputed
#     #     s = np.zeros(self.k)                                   # s[c] = half the distance to closest other centroid
#     #     d = np.full((self.k, self.k), np.inf)                  # centroid distance matrix
#     #     m = np.zeros_like(self.centroids)                      # new centroids (mean)

#     #     # Set initial bounds
#     #     for x in range(self.n_samples):
#     #         for c in range(self.k):
#     #             self.l[x, c] = self.dist(self.data[x], self.centroids[c])
#     #         self.u[x] = self.l[x, self.y_pred[x]]

#     #     while True:
#     #         self.iterations += 1
#     #         old_centroids = self.centroids.copy()
#     #         old_pred = self.y_pred.copy()

#     #         debug and print('\nNew iteration')

#     #         # 1. Compute centroid distances and s[c]
#     #         for c in range(self.k):
#     #             for cprime in range(c + 1, self.k):
#     #                 dval = self.dist(self.centroids[c], self.centroids[cprime])
#     #                 d[c, cprime] = d[cprime, c] = dval
#     #             s[c] = 0.5 * np.min([d[c, j] for j in range(self.k) if j != c])

#     #         # 2-3. Reassign points
#     #         for x in range(self.n_samples):
#     #             assigned = self.y_pred[x]

#     #             if self.u[x] <= s[assigned]:
#     #                 continue

#     #             if r[x]:
#     #                 current_distance = self.dist(self.data[x], self.centroids[assigned])
#     #                 self.u[x] = current_distance
#     #                 r[x] = False
#     #             else:
#     #                 current_distance = self.u[x]

#     #             for c in range(self.k):
#     #                 if c == assigned:
#     #                     continue

#     #                 if self.u[x] <= self.l[x, c] or self.u[x] <= 0.5 * d[assigned, c]:
#     #                     continue  # Triangle inequality allows skipping

#     #                 # Compute distance and update lower bound
#     #                 candidate_distance = self.dist(self.data[x], self.centroids[c])
#     #                 self.l[x, c] = candidate_distance

#     #                 if candidate_distance < current_distance:
#     #                     self.y_pred[x] = c
#     #                     current_distance = candidate_distance
#     #                     self.u[x] = candidate_distance

#     #         # 4. Compute new centroids
#     #         for c in range(self.k):
#     #             mask = self.y_pred == c
#     #             if np.any(mask):
#     #                 m[c] = np.mean(self.data[mask], axis=0)

#     #         # 5. Update lower bounds
#     #         for x in range(self.n_samples):
#     #             for c in range(self.k):
#     #                 shift = self.dist(old_centroids[c], m[c])
#     #                 self.l[x, c] = max(self.l[x, c] - shift, 0)

#     #         # 6. Update upper bounds and mark r[x]
#     #         for x in range(self.n_samples):
#     #             assigned = self.y_pred[x]
#     #             self.u[x] += self.dist(old_centroids[assigned], m[assigned])
#     #             r[x] = True

#     #         # 7. Update centroids
#     #         for c in range(self.k):
#     #             if np.any(self.y_pred == c):
#     #                 self.centroids[c] = m[c]

#     #         # Check for convergence
#     #         if np.all(old_pred == self.y_pred):
#     #             break



#     # v1
#     # def _elkan(self, debug=0):

#     #     r = np.full(self.n_samples, False)
#     #     s = np.zeros(self.k)
#     #     d = np.full((self.k, self.k), np.inf)
#     #     m = np.zeros((self.k, self.data.shape[1]))

        
#     #     old_pred = np.zeros(self.n_samples, dtype=int)
#     #     np.copyto(old_pred, self.y_pred)
#     #     old_centroids = np.zeros((self.k, self.data.shape[1]))
#     #     np.copyto(old_centroids, self.centroids)
#     #     dist_change = np.zeros(self.k)

#     #     self.centroids = self._move_centroids()

#     #     for c in range(self.k):
#     #         dist_change = self.dist(self.centroids[c], old_centroids[c])            
#     #         for x in range(self.n_samples):
#     #             self.l[x, c] -= dist_change
#     #             self.u[x] += dist_change

#     #     while True:
#     #         self.iterations += 1
#     #         old_centroids = self.centroids.copy()
#     #         old_pred = self.y_pred.copy()

#     #         debug and print('\nNew iteration')
            

#     #         # 1. distance matrix of centers
#     #         for c in range(self.k):
#     #             for cprime in range(c, self.k):
#     #                 d[c, cprime] = d[cprime, c] = self.dist(self.centroids[c], self.centroids[cprime])

#     #             # s is the min over c' different from c
#     #             s[c] = 0.5 * np.min(d[c, [i for i in range(self.k) if i != c]])

#     #         for x in range(self.n_samples):
#     #             # 2. identify all points such that u(x) <= s(c(x))
#     #             if self.u[x] <= s[self.y_pred[x]]:
#     #                 continue

#     #             else:
#     #                 for c in range(self.k):
#     #                     # 3.
#     #                     if c != self.y_pred[x] and self.u[x] > self.l[x, c] and self.u[x] > 0.5 * d[self.y_pred[x], c]:

#     #                         # 3a.
#     #                         if r[x]:
#     #                             self._distance_cache[x, self.y_pred[x]] = self.dist(self.data[x], self.centroids[self.y_pred[x]])
#     #                             r[x] = False
#     #                         else:
#     #                             self._distance_cache[x, self.y_pred[x]] = self.u[x]

#     #                         # 3b.
#     #                         if self._distance_cache[x, self.y_pred[x]] > self.l[x, c] or self._distance_cache[x, self.y_pred[x]] > 0.5 * d[self.y_pred[x], c]:
#     #                             if self.dist(self.data[x], self.centroids[c]) < self._distance_cache[x, self.y_pred[x]]:
#     #                                 self.y_pred[x] = c
                        
#     #         # 4. for each center, caclulate the mean of points assigned to it
#     #         for c in range(self.k):
#     #             mask = self.y_pred == c
#     #             points_in_cluster = self.data[mask]

#     #             if np.any(mask):
#     #                 # Vectorized computation
#     #                 m[c] = np.mean(points_in_cluster, axis=0)

#     #         # 5.
#     #         for x in range(self.n_samples):
#     #             for c in range(self.k):
#     #                 self.l[x, c] = max(self.l[x, c] - self.dist(old_centroids[c], m[c]), 0)
#     #             self.u[x] += self.dist(old_centroids[self.y_pred[x]], m[self.y_pred[x]])
#     #             r[x] = True

#     #         # 6.
#     #         for x in range(self.n_samples):
#     #             self.u[x] += self.dist(m[self.y_pred[x]], self.centroids[self.y_pred[x]])
#     #             r[x] = True

#     #         # 7. replace each center c by m[c]
#     #         for c in range(self.k):
#     #             if np.any(self.y_pred == c):
#     #                 self.centroids[c] = m[c]

#     #         # check for convergence
#     #         if np.all(old_pred == self.y_pred):
#     #             break

    
#     def _acc_hartigan(self, debug=0):
#         self.centroids = self._move_centroids(None, debug > 1)
#         cluster_sizes = np.bincount(self.y_pred, minlength=self.k)

#         # Initialize bounds
#         u = np.zeros(self.n_samples)  # upper bounds
#         l = np.zeros((self.n_samples, self.k))  # lower bounds
#         d_cc = np.full((self.k, self.k), np.inf)  # centroid- centroid distances

#         for i in range(self.n_samples):
#             u[i] = self.dist(self.data[i], self.centroids[self.y_pred[i]])
#             for c in range(self.k):
#                 if c != self.y_pred[i]:
#                     l[i, c] = max(0, self.dist(self.data[i], self.centroids[c]))

#         edit = True
#         while edit:
#             self.iterations += 1
#             edit = False

#             # Update centroid-centroid distances and s[c]
#             s = np.full(self.k, np.inf)
#             for i in range(self.k):
#                 for j in range(i + 1, self.k):
#                     d_cc[i, j] = d_cc[j, i] = self.dist(self.centroids[i], self.centroids[j])
#             for c in range(self.k):
#                 s[c] = 0.5 * np.min([d_cc[c, j] for j in range(self.k) if j != c])

#             for i in range(self.n_samples):
#                 curr = self.y_pred[i]

#                 if u[i] <= s[curr]:
#                     continue  # Skip this point; it cannot move to any better cluster

#                 best_cost_reduction = 0
#                 best_target = None

#                 for c in range(self.k):
#                     if c == curr:
#                         continue
#                     if u[i] <= l[i, c]:
#                         continue  # Prune using lower bound

#                     # Hartigan cost difference
#                     cost_out = -cluster_sizes[curr] / (cluster_sizes[curr] - 1) * self.dist(self.data[i], self.centroids[curr])
#                     cost_in = cluster_sizes[c] / (cluster_sizes[c] + 1) * self.dist(self.data[i], self.centroids[c])
#                     delta_cost = cost_out + cost_in

#                     if delta_cost < best_cost_reduction:
#                         best_cost_reduction = delta_cost
#                         best_target = c

#                 if best_target is not None:
#                     old = curr
#                     new = best_target

#                     self.y_pred[i] = new
#                     cluster_sizes[old] -= 1
#                     cluster_sizes[new] += 1

#                     # Update centroids
#                     self.centroids[old] = np.mean(self.data[self.y_pred == old], axis=0)
#                     self.centroids[new] = np.mean(self.data[self.y_pred == new], axis=0)

#                     # Update bounds for i
#                     u[i] = self.dist(self.data[i], self.centroids[new])
#                     for c in range(self.k):
#                         if c != new:
#                             l[i, c] = max(l[i, c], self.dist(self.data[i], self.centroids[c]))

#                     edit = True


#     def _move_centroids(self, move_just = None, debug=0):
#         """
#         Move the centroids to the mean of their cluster.
#         """

#         debug and print('\n  moving centroids...')
#         debug and print('  | y_pred:', self.y_pred)
#         debug and print('  | data:\n', self.data)
#         debug and print('  | centroids_before:\n', self.centroids)

#         centroids = np.copy(self.centroids)
        
#         move = move_just if move_just is not None else range(self.k)
#         debug and print('  | move:', move)
        
#         for centroid_id in move:
            
#             mask = self.y_pred == centroid_id
            
#             # if centroid has no points assigned to it, reassign it randomly
#             if np.any(mask):
#                 centroids[centroid_id] = np.mean(self.data[mask], axis=0)
#             else:
#                 debug and print(f"  Centroid {centroid_id} is empty. Reassigning.")
#                 new_centroid_id = np.random.choice(self.n_samples)
#                 centroids[centroid_id] = self.data[new_centroid_id]
#                 self.y_pred[new_centroid_id] = centroid_id

#         debug and print('  centroids_after:\n', centroids)

#         return centroids


#     def _assign_clusters(self, specific_centroids=None, debug=0):
#         """
#         Assign each data point to the closest centroid.

#         Parameters
#         ----------
#         specific_centroids : np.ndarray
#             array of shape (k', d) with specific centroids (of a specific number k') to use for the assignment

#         Returns
#         -------
#         np.ndarray
#             Array of length n with cluster assignments for each sample
#         """

#         if specific_centroids is None:
#             specific_centroids = self.centroids
        
#         self._distance_cache = cdist(self.data, specific_centroids, metric='sqeuclidean')
#         y_pred = np.argmin(self._distance_cache, axis=1)
#         self.norm_calculations += specific_centroids.shape[0] / (self.k*self.n_samples)

#         # update upper and lower bounds
#         # set l(x,c) = d(x,c)
#         for i in range(self.n_samples):
#             for j in range(self.k):
#                 self.l[i,j] = self._distance_cache[i,j]

#         self.u = np.zeros(self.n_samples)
#         for i in range(self.n_samples):
#             self.u[i] = np.min(self._distance_cache[i])
        
#         debug and print('y_pred:', y_pred)

#         return y_pred


#     def _tot_cluster_cost(self, centroids, points_ids, debug=0):
#         """
#         Compute the overall cost of clustering

#         Parameters
#         ----------
#         centroids : np.ndarray
#             Array of shape ('k, d) with cluster centroids
#         points_ids : np.ndarray
#             Array of length n with cluster assignments for each sample

#         Returns
#         -------
#         float
#             Total cost of clustering
#         """
        
#         debug and print('\n  calculating _tot_cluster_cost')
        
#         total_cost = 0

#         for centroid_id in range(centroids.shape[0]):
#             mask = points_ids == centroid_id
#             points_in_cluster = self.data[mask]

#             cluster_items = np.where(points_ids == centroid_id)[0]

#             if len(points_in_cluster) > 0:
#                 # Vectorized computation
#                 cost = np.sum(np.sum((points_in_cluster - centroids[centroid_id])**2, axis=1))
#                 total_cost += cost
#                 self.norm_calculations += len(cluster_items) / (self.k*self.n_samples)

#             debug and print('  | centroid_id:', centroid_id)
#             debug and print('  | centroid:', centroids[centroid_id])
#             debug and print('  | cluster_items:', cluster_items)
#             debug and print('  | cost:', cost)
        
#         debug and print(f'  total_cost: {total_cost}')
        
#         return total_cost


#     def _find_candidates(self, datapoint_id, candidates, debug=0):
#         """
#         Find candidates for reassignment of a single datapoint.
#         """

#         # calculate sizes of clusters and cost of current assignment
#         cluster_sizes = np.bincount(self.y_pred, minlength=self.k)

#         current_centroid_id = self.y_pred[datapoint_id]
#         current_size = cluster_sizes[current_centroid_id]
        
#         # if current clsuter has only one point, we cannot move it
#         if current_size <= 1:
#             return candidates
        
#         prefactor = current_size / (current_size - 1)
#         current_cost = prefactor * np.sum((self.data[datapoint_id] - self.centroids[current_centroid_id])**2)
#         self.norm_calculations += 1/(self.k*self.n_samples)

#         debug and print('current_cost:', current_cost)

#         # if current_cost is 0, delta_cost will always be positive
#         if current_cost == 0:
#             return candidates
        
#         # Vectorized computation for all centroids except the current one
#         mask = np.arange(self.k) != current_centroid_id
#         cluster_sizes_masked = cluster_sizes[mask]
#         valid_centroid_ids = np.where(mask)[0]

#         # Compute delta costs
#         delta_costs = (cluster_sizes_masked / (cluster_sizes_masked + 1)) * np.linalg.norm(self.data[datapoint_id] - self.centroids[valid_centroid_ids], axis=1)**2 - current_cost
#         self.norm_calculations += (len(cluster_sizes_masked)) / (self.k*self.n_samples)

#         # Find the best centroid (most negative delta_cost)
#         best_idx = np.argmin(delta_costs)
#         best_delta_cost = delta_costs[best_idx]

#         if best_delta_cost < 0:
#             best_centroid_id = valid_centroid_ids[best_idx]
#             candidates[datapoint_id] = [best_delta_cost, current_centroid_id, best_centroid_id]
        
#         return candidates


#     def _accept_candidates(self, candidates, debug=0):
#         """
#         Accepts all candidates passed as argument and calculates new total cluster cost.
#         """
#         # accept all candidates
#         used_centroids = set()

#         for datapoint_id, [_, current_centroid_id, new_centroid_id] in candidates.items():

#             debug and print('candidate:', datapoint_id)

#             used_centroids.add(current_centroid_id)
#             used_centroids.add(new_centroid_id)

#             debug and print('y_pred before:', self.y_pred)

#             # update closest_points_ids assigning datapoint to new_centroid_id
#             self.y_pred[datapoint_id] = new_centroid_id
#             debug and print('y_pred after:', self.y_pred)

#         new_centroids = self._move_centroids(move_just=used_centroids, debug = debug)

#         return self._tot_cluster_cost(new_centroids, self.y_pred, debug), new_centroids